In [1]:
import pandas as pd
import numpy as np
import json
from scipy.spatial.transform import Rotation as R

In [2]:
input_curves = pd.read_table('curves.json',header=None).iloc[0,0]

In [3]:
jdata = json.loads(input_curves)

In [4]:
len(jdata)

236

In [5]:
list_camera = list()
flag_update = False
for i in range(len(jdata)):
    x_i = jdata[i]
    dict_camera_i = x_i.copy()
    
    if(0<i<(len(jdata)-1)):
        x_i_prev = jdata[i-1]
        x_i_next = jdata[i+1]
        if(x_i_prev['branch_id'] == x_i['branch_id'] == x_i_next['branch_id']):
            vec_fw = np.array([x_i_next['x'] - x_i['x'],x_i_next['y'] - x_i['y'],x_i_next['z'] - x_i['z']])
            mag_fw = np.linalg.norm(vec_fw)
            dict_camera_i['rotation_fw'] = [np.arccos(vec_fw[0]/mag_fw)*180/np.pi,
                                            np.arccos(vec_fw[1]/mag_fw)*180/np.pi,
                                            np.arccos(vec_fw[2]/mag_fw)*180/np.pi]

            vec_bw = np.array([x_i_prev['x'] - x_i['x'],x_i_prev['y'] - x_i['y'],x_i_prev['z'] - x_i['z']])
            mag_bw = np.linalg.norm(vec_bw)
            dict_camera_i['rotation_bw'] = [np.arccos(vec_bw[0]/mag_bw)*180/np.pi,
                                            np.arccos(vec_bw[1]/mag_bw)*180/np.pi,
                                            np.arccos(vec_bw[2]/mag_bw)*180/np.pi]
            if(flag_update):
                list_camera[i-1]['rotation_bw'] = dict_camera_i['rotation_bw']
                flag_update = False
        elif(x_i_prev['branch_id'] != x_i['branch_id']):
            vec_fw = np.array([x_i_next['x'] - x_i['x'],x_i_next['y'] - x_i['y'],x_i_next['z'] - x_i['z']])
            mag_fw = np.linalg.norm(vec_fw)
            dict_camera_i['rotation_fw'] = [np.arccos(vec_fw[0]/mag_fw)*180/np.pi,
                                            np.arccos(vec_fw[1]/mag_fw)*180/np.pi,
                                            np.arccos(vec_fw[2]/mag_fw)*180/np.pi]
            flag_update = True
            
        elif(x_i_next['branch_id'] != x_i['branch_id']):
            vec_bw = np.array([x_i_prev['x'] - x_i['x'],x_i_prev['y'] - x_i['y'],x_i_prev['z'] - x_i['z']])
            mag_bw = np.linalg.norm(vec_bw)
            dict_camera_i['rotation_bw'] = [np.arccos(vec_bw[0]/mag_bw)*180/np.pi,
                                            np.arccos(vec_bw[1]/mag_bw)*180/np.pi,
                                            np.arccos(vec_bw[2]/mag_bw)*180/np.pi]  
            dict_camera_i['rotation_fw'] = list_camera[i-1]['rotation_fw']
    if(i==0):
        x_i_next = jdata[i+1]
        vec_fw = np.array([x_i_next['x'] - x_i['x'],x_i_next['y'] - x_i['y'],x_i_next['z'] - x_i['z']])
        mag_fw = np.linalg.norm(vec_fw)
        dict_camera_i['rotation_fw'] = [np.arccos(vec_fw[0]/mag_fw)*180/np.pi,
                                        np.arccos(vec_fw[1]/mag_fw)*180/np.pi,
                                        np.arccos(vec_fw[2]/mag_fw)*180/np.pi]
        flag_update = True
    if(i==len(jdata)-1):
        x_i_prev = jdata[i-1]
        vec_bw = np.array([x_i_prev['x'] - x_i['x'],x_i_prev['y'] - x_i['y'],x_i_prev['z'] - x_i['z']])
        mag_bw = np.linalg.norm(vec_bw)
        dict_camera_i['rotation_bw'] = [np.arccos(vec_bw[0]/mag_bw)*180/np.pi,
                                        np.arccos(vec_bw[1]/mag_bw)*180/np.pi,
                                        np.arccos(vec_bw[2]/mag_bw)*180/np.pi]  
        dict_camera_i['rotation_fw'] = list_camera[i-1]['rotation_fw']    
    list_camera.append(dict_camera_i)

In [6]:
list_camera[:10]

[{'branch_id': 'S3_S5',
  'pseudotime': 0.0,
  'x': -0.06593725135199813,
  'y': -0.022338808438985724,
  'z': -0.046025399083270205,
  'rotation_fw': [66.03768407500867, 73.27744505825859, 29.85019394126952],
  'rotation_bw': [113.96231592499134, 106.72255494174142, 150.1498060587305]},
 {'branch_id': 'S3_S5',
  'pseudotime': 0.0019271269610602309,
  'x': -0.06515457628296169,
  'y': -0.021784301644704976,
  'z': -0.04435394454441773,
  'rotation_fw': [66.03768407500867, 73.27744505825859, 29.85019394126952],
  'rotation_bw': [113.96231592499134, 106.72255494174142, 150.1498060587305]},
 {'branch_id': 'S3_S5',
  'pseudotime': 0.0038542539221204618,
  'x': -0.06437190121392525,
  'y': -0.021229794850424227,
  'z': -0.04268249000556526,
  'rotation_fw': [66.03768407500867, 73.27744505825859, 29.85019394126952],
  'rotation_bw': [113.96231592499134, 106.72255494174142, 150.1498060587305]},
 {'branch_id': 'S3_S5',
  'pseudotime': 0.005781380883180693,
  'x': -0.06358922614488881,
  'y': -

In [7]:
with open('curves_with_angles.json', 'w') as f:
    json.dump(list_camera, f)